# Homework 3

In this notebook we will present our solution for the third homework of ADA.

In [2]:
#Importing all the libraries
import pandas as pd
import numpy as np
import folium as fl

We begin by importing the whole CSV thanks to the pandas library.

In [42]:
path = 'Data/P3_GrantExport.csv'

#Reading the csv with Pandas

data = pd.read_csv(path,sep = ';')

We first take a look at the data!

In [43]:
data.head()

,Project Number,Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
2,5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
3,6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
4,7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN


We first begin by looking at the "Approved Amount" filed, which contains the amount the is given to the university.

In [76]:
data['Approved Amount'].value_counts().head()

data not included in P3    10910
10000.00                     611
5000.00                      348
8000.00                      320
15000.00                     273
Name: Approved Amount, dtype: int64

We can see that for 1/6 of the funding there is no amount that is registered, therefore we can remove them from the database as we will not use them for the sum of grant money in each canton.

In [85]:
data_clean = data[data['Approved Amount'].isin(['data not included in P3']) == False]

By reading the [SNF](http://p3.snf.ch/Pages/DataAndDocumentation.aspx) page we also find that if the research is not carried out at a swiss institution, the 'University' field will be left blank. As we are not interested in funding that goes outside of Switzerland we can also delete these rows.

In [86]:
swissFunding = data_clean.dropna(subset=['University'])

As we can see that made us drop a large part of the data already.

In [87]:
print('Initial size of data : ', len(data))
print('Usable data for switzerland : ',len(swissFunding))
print('Percentage of Data kept',round(len(swissFunding)/len(data),3)*100,' %')

Initial size of data :  63969
Usable data for switzerland :  50878
Percentage of Data kept 79.5  %


For this analysis, we are only interested in the University name and the funding amount so that we can sort by cantons, we will therefore drop the other columns. We also change the type of the approved amount to float so that we can use them as numbers afterwards.

In [93]:
swissFunding = swissFunding[['University','Approved Amount']]
swissFunding['Approved Amount'] = swissFunding['Approved Amount'].astype(np.float)
swissFunding.dtypes

University          object
Approved Amount    float64
dtype: object

We then proceed to group by university.

In [94]:
uniGroup = swissFunding.groupby('University')

We then use the aggregate sum function to get the total funding per university.

In [102]:
fundingUni = uniGroup.sum()
fundingUni.head()

,Approved Amount
University,
AO Research Institute - AORI,3435621.0
Allergie- und Asthmaforschung - SIAF,19169965.0
Berner Fachhochschule - BFH,31028695.0
Biotechnologie Institut Thurgau - BITG,2492535.0
Centre de rech. sur l'environnement alpin - CREALP,1567678.0
